In [1]:
import pandas as pd
import panel as pn
import holoviews as hv
import numpy as np
import panel as pn

def interactive_plot(data_frame):
    # (1) Check if DataFrame has at least 3 columns
    if len(data_frame.columns) < 3:
        raise ValueError("The DataFrame should have at least 3 columns. Use a different function for this data.")

    # Reshape DataFrame to multi-dimensional array for easier indexing
    df_shape = [len(data_frame.index)] + [len(level) for level in data_frame.columns.levels]
    data = data_frame.values.reshape(df_shape)

    # (2) & (3) Define plotting functions
    def plot_1D(*indices):
        idx = [slice(None) if i < 2 else indices[i-2] for i in range(len(df_shape))]
        print(f'idx={idx}', flush=True)
        return hv.Curve(data[tuple(idx)])

    def plot_2D(*indices):
        idx = [slice(None) if i < 2 else indices[i-3] for i in range(len(df_shape))]
        return hv.Image(data[tuple(idx)], kdims=['Dim 0', 'Dim 1'])

    # (4) Create widgets
    dim_selector = pn.widgets.RadioBoxGroup(name='Dimension', options=['1D', '2D'], inline=True)

    index_names = data_frame.columns.names
    print(index_names)

    sliders = [pn.widgets.IntSlider(name=index_names[i], start=0, end=df_shape[i]-1) for i in range(len(index_names))]

    special_dim_0 = pn.widgets.IntInput(name='Special Dimension 0', value=0, step=1)
    special_dim_1 = pn.widgets.IntInput(name='Special Dimension 1', value=1, step=1)

    @pn.depends(special_dim_0.param.value, special_dim_1.param.value)
    def update_slider_names(special_dim_0_value, special_dim_1_value):
        for i, slider in enumerate(sliders):
            dim_idx = i + 2  # Adjust index to skip the first two dimensions
            if dim_idx == special_dim_0_value:
                slider.name = f'Special Dim 0: {index_names[dim_idx]}'
            elif dim_idx == special_dim_1_value:
                slider.name = f'Special Dim 1: {index_names[dim_idx]}'
            else:
                slider.name = f'{index_names[dim_idx]}'

    @pn.depends(dim_selector.param.value, *sliders, special_dim_0.param.value, special_dim_1.param.value)
    def reactive_plot(dim, *indices, special_dim_0=0, special_dim_1=1):
        # Update the index positions based on special dimensions
        if dim == '1D':
            indices = indices[:special_dim_0] + (slice(None),) + indices[special_dim_0:special_dim_1] + (slice(None),) + indices[special_dim_1:]
            print(f'indices1D={indices}', flush=True)
            return plot_1D(*indices)
        else:
            indices = indices[:special_dim_0] + (slice(None), slice(None)) + indices[special_dim_0:]
            print(f'indices2D={indices}', flush=True)
            return plot_2D(*indices)

    # (5) Create layout and return
    layout = pn.Row(
        pn.Column(dim_selector, *sliders, special_dim_0, special_dim_1, update_slider_names),
        reactive_plot
    )
    return layout

# Example usage:
# Assume df is your DataFrame
# layout = interactive_plot(df)
# layout.servable()  # or layout.show()

In [54]:
def iplot(data_frame):
    # (1) Check if DataFrame has at least 3 columns
    if len(data_frame.columns) < 3:
        raise ValueError("The DataFrame should have at least 3 columns. Use a different function for this data.")

    if len(data_frame) != 1:
        raise ValueError("Data frame must have only one row. Please reshape your data accordingly.")

    # Reshape DataFrame to multi-dimensional array for easier indexing
    df_shape = [len(data_frame.index)] + [len(level) for level in data_frame.columns.levels]
    df_shape = df_shape[1:]
    data = data_frame.values.reshape(df_shape)

    index_names = data_frame.columns.names

    # (2) & (3) Define plotting functions
    def plot_1D(*indices):
        # print(f'indices={indices}')
        # idx = [slice(None) if i < 2 else indices[i-2] for i in range(len(df_shape))]
        # print(f'idx={idx}', flush=True)
        return hv.Curve(data[tuple(indices)])

    def plot_2D(*indices, kdims):
        return hv.Image(data[tuple(indices)], kdims=kdims)

    # (4) Create widgets
    dim_selector = pn.widgets.RadioBoxGroup(name='Dimension', options=['1D', '2D'], inline=True)

    sliders = [pn.widgets.IntSlider(name=index_names[i], start=0, end=df_shape[i]-1) for i in range(len(index_names))]

    special_dim_0 = pn.widgets.IntInput(name='Special Dimension 0', value=0, step=1)
    special_dim_1 = pn.widgets.IntInput(name='Special Dimension 1', value=1, step=1)

    @pn.depends(special_dim_0.param.value, special_dim_1.param.value)
    def update_slider_names(special_dim_0_value, special_dim_1_value):
        for dim_idx, slider in enumerate(sliders):
            if dim_idx == special_dim_0_value:
                slider.name = f'{index_names[dim_idx]} (IGNORE SLIDER -- Plot Axis 0)'
            elif dim_idx == special_dim_1_value:
                slider.name = f'{index_names[dim_idx]} (IGNORE SLIDER in 2D MODE -- Plot Axis 1)'
            else:
                slider.name = f'{index_names[dim_idx]}'

    # @pn.depends(dim_selector.param.value, *sliders, special_dim_0.param.value, special_dim_1.param.value)
    # def reactive_plot(dim, *indices, special_dim_0, special_dim_1):
    #     print(f'INDICES={indices}')
    #     # Update the index positions based on special dimensions
    #     if dim == '1D':
    #         indices = indices[:special_dim_0] + (slice(None),) + indices[special_dim_0:special_dim_1] + (slice(None),) + indices[special_dim_1:]
    #         print(f'indices1D={indices}', flush=True)
    #         return plot_1D(*indices)
    #     else:
    #         indices = indices[:special_dim_0] + (slice(None), slice(None)) + indices[special_dim_0:]
    #         print(f'indices2D={indices}', flush=True)
    #         return plot_2D(*indices)

    @pn.depends(dim_selector.param.value, *sliders, special_dim_0.param.value, special_dim_1.param.value)
    def reactive_plot(dim, *indices):
        dim = 1 if dim == '1D' else 2
        special_dims = indices[-2:]
        indices = indices[:-2]
        plot_method = plot_1D if dim == 1 else lambda *x : plot_2D(*x, kdims=[index_names[special_dims[0]], index_names[special_dims[1]]])
        if( special_dims[0] == special_dims[1] and dim == 2 ):
            print(f'Plotted dimensions are the same, dimension = {special_dims[0]}, no update to plot taken', end='\r')
            return
        idx = [slice(None) if i in special_dims[:dim] else indices[i] for i in range(len(indices))]
        return plot_method(*idx)

    # (5) Create layout and return
    layout = pn.Row(
        pn.Column(dim_selector, *sliders, special_dim_0, special_dim_1, update_slider_names),
        reactive_plot
    )
    return layout

# Example usage:
# Assume df is your DataFrame
# layout = interactive_plot(df)
# layout.servable()  # or layout.show()

In [55]:
pn.extension()
np.random.seed(0)  # Seed for reproducibility
data_shape = (1, 50, 50, 30, 20)  # Shape of the multi-dimensional data
random_data = np.random.randn(*data_shape) 

# Create a DataFrame with a multi-level column index
columns = pd.MultiIndex.from_product([range(dim_size) for dim_size in data_shape[1:]],
                                     names=[f'My CUSTOM name {i}' for i in range(1,len(data_shape))])
df = pd.DataFrame(random_data.reshape(data_shape[0], -1), columns=columns)

mechs = [interactive_plot, iplot]
mech = mechs[1]
# Create interactive plot layout
layout = iplot(df)

# Display the layout
layout.servable()

Row
    [0] Column
        [0] RadioBoxGroup(inline=True, name='Dimension', options=['1D', '2D'], value='1D')
        [1] IntSlider(end=49, name='My CUSTOM name 1...)
        [2] IntSlider(end=49, name='My CUSTOM name 2...)
        [3] IntSlider(end=29, name='My CUSTOM name 3')
        [4] IntSlider(end=19, name='My CUSTOM name 4')
        [5] IntInput(name='Special Dimension 0')
        [6] IntInput(name='Special Dimension 1', value=1)
        [7] ParamFunction(function, _pane=Str, defer_load=False)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)